In [ ]:
import dva_sdhdf_combine_v3
import imp
import os
import subprocess
import h5py
import numpy as np
from astropy.time import Time
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import datetime
import matplotlib.dates as mdates
from matplotlib.dates import HourLocator as HourLocator
from matplotlib.dates import MinuteLocator as MinuteLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy import units as u
from astropy.time import TimeDelta
from astropy.modeling import models, fitting

In [ ]:
#day ='41'
#surveyphase = '2'

#### Change the directory to where the files are located" ####
#dir_in = '/home2/DATA_AO/DVA_DATA/survey_phase2_day'+day+'/'
#dir_out = '/home2/DATA_AO/DVA_DATA/dva_phase2/survey_phase2_day'+day+'/'
#dir_in = '/home2/DATA_AO/DVA_DATA/drone_CHIME/'
#dir_out = '/home2/DATA_AO/DVA_DATA/drone_CHIME/'
#dir_in = '/home/ordoga/Mar29_spect_test/test_Mar29/'
#dir_out = '/home/ordoga/Mar29_spect_test/test_Mar29/'
dir_in = '/home2/DATA_AO/DVA_DATA/el_scans_Apr2023/'
dir_out = '/media/ordoga/15m_band1_survey/el_scans_Apr2023/'
##############################################################

In [ ]:
imp.reload(dva_sdhdf_combine_v3)

#start = '2022-10-26T23:30:00.0'
#stop = '2022-10-27T01:20:00.0'
#outname = 'drone_CHIME'

start = '2023-04-03T21:24:00.0'
stop = '2023-04-03T23:46:07.0'
outname = 'el_scans_Apr2023'

dva_sdhdf_combine_v3.combine(dir_in,dir_out,start,stop,outname,freq_s=2,
                                 az_scan_trim=False,freq_avg=True)

## Read in data

In [ ]:
%%time

#outname = 'Day41_fail'
#outname = 'drone_CHIME'
outname = 'el_scans_Apr2023'
file = h5py.File(dir_out+outname+'.h5','r')
print(file)
    
dataset = file['data']['beam_0']['band_SB0']['scan_0']
    
# Add the position and time data to the corresponding arrays:
dec_set = dataset['metadata']['declination']
ra_set = dataset['metadata']['right_ascension']
el_set = dataset['metadata']['elevation']
az_set = dataset['metadata']['azimuth']
t_set = dataset['metadata']['utc']
noise_set = dataset['metadata']['noise_state']
freq = file['data']['beam_0']['band_SB0']['frequency'][:]/1e6
RR_set = dataset['data'][:,0,:]
LL_set = dataset['data'][:,1,:]
reRL_set = dataset['data'][:,2,:]
imRL_set = dataset['data'][:,3,:]
    
t_set_plt = Time(t_set, format='isot',scale='utc').mjd

## Generic waterfall plot

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(15,10)) 
    
fs = 12
    
extent = [t_set_plt[0],t_set_plt[-1],freq[0],freq[-1]]
    
im = axs.imshow(10*np.log10(RR_set.T),aspect='auto',vmin=65,vmax=85,
                origin='lower',extent=extent,cmap='viridis')

divider = make_axes_locatable(axs)
cax = divider.append_axes('right', size='2%', pad=0.05)
cbar = fig.colorbar(im, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=fs) 
cbar.set_label('Power (dB)', fontsize=fs)

axs.set_xlim(extent[0],extent[1])
axs.set_ylim(freq[0],freq[-1])
axs.tick_params(axis='both', labelsize=fs)
axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
axs.fmt_xdata = mdates.DateFormatter('%H:%M:%S')
axs.set_xlabel('Time (UTC)',fontsize=fs)        
axs.set_ylabel('Frequency (MHz)',fontsize=fs)

#plt.tight_layout()
#plt.savefig('../DVA2/PLOTS/dva_drone_noise_waterfall.pdf')

In [ ]:
fig1, axs = plt.subplots(1,1,figsize=(18,8))

i1 = 0
i2 = 8000
fr_idx = 5500
print(freq[fr_idx])
print(LL_set[:,fr_idx].shape)
print(t_set_plt.shape)
print(t_set_plt)

axs.plot(t_set_plt[i1:i2],10*np.log10(LL_set[i1:i2,fr_idx]),color='black')
ax2 = axs.twinx()
ax2.plot(t_set_plt[i1:i2],az_set[i1:i2])
ax2.plot(t_set_plt[i1:i2],el_set[i1:i2])
#axs.legend(fontsize=14)

axs.tick_params(axis="x")
axs.tick_params(axis="y")
#axs.set_xlim(350,1030)
axs.set_ylim(70,80)
#axs.set_xlabel('Frequency (MHz)',fontsize=14)
#axs.set_ylabel('Power (dB)',fontsize=14)
#axs.set_title('Survey phase '+surveyphase+' day '+day,fontsize=14)
axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
axs.fmt_xdata = mdates.DateFormatter('%H:%M:%S')
axs.set_xlabel('Time (UTC)',fontsize=fs)  
axs.grid()

In [ ]:
fig1, axs = plt.subplots(1,1,figsize=(18,6))

ii1 = 1600
ii2 = 2850

print(t_set[ii1])
print(t_set[ii2])

axs.plot(freq,10*np.log10(LL_set[ii1,:]),label='with drone')
axs.plot(freq,10*np.log10(LL_set[ii2,:]),label='no drone')
axs.legend(fontsize=14)

axs.tick_params(axis="x")
axs.tick_params(axis="y")
axs.set_xlim(350,1030)
axs.set_ylim(65,95)
axs.set_xlabel('Frequency (MHz)',fontsize=14)
axs.set_ylabel('Power (dB)',fontsize=14)
#axs.set_title('Survey phase '+surveyphase+' day '+day,fontsize=14)
axs.grid()

#plt.tight_layout()
#plt.savefig('../DVA2/PLOTS/dva_drone_noise_spectrum.pdf')

In [ ]:
10**(-0.25/10)

In [ ]:
fig1, axs = plt.subplots(1,1,figsize=(18,6))

axs.scatter(t_set_plt,az_set,s=1)

axs.tick_params(axis="x")
axs.tick_params(axis="y")
axs.set_xlim(t_set_plt[0]+0.08,t_set_plt[-1]+0.01)
axs.set_ylim(-10,370)
axs.set_xlabel('Time (UTC)',fontsize=14)
#axs.set_ylabel('RA (hr)',fontsize=14)
axs.set_title('Survey phase '+surveyphase+' day '+day,fontsize=14)
axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
axs.fmt_xdata = mdates.DateFormatter('%H:%M:%S')
axs.grid()

In [ ]:
print(az_set[-1])
print(t_set[-1])

In [ ]:
(283.77-67)*18/360

In [ ]:
imp.reload(dva_sdhdf_combine_v3)

start = '2022-09-26T00:00:00.0'
stop = '2022-09-26T23:59:59.0'

#start = '2022-09-24T00:31:58.0'
#stop = '2022-09-26T14:33:58.0'
outname = 'NCP_Fall_2022_part3'
dirfiles = '/home2/DATA_AO/DVA_DATA/NCP_Sept2022/'

dva_sdhdf_combine_v3.combine(dirfiles,dirfiles,start,stop,outname,freq_s=12,
                                 az_scan_trim=False,freq_avg=True)

In [ ]:
%%time
dirfiles = '/home2/DATA_AO/DVA_DATA/NCP_Sept2022/'
outname = 'NCP_Fall_2022_part3'
file = h5py.File(dirfiles+outname+'.h5','r')
print(file['data']['beam_0'].keys())
    
dataset = file['data']['beam_0']['band_SB0']['scan_0']
    
# Add the position and time data to the corresponding arrays:
dec_set = dataset['metadata']['declination']
ra_set = dataset['metadata']['right_ascension']
el_set = dataset['metadata']['elevation']
az_set = dataset['metadata']['azimuth']
t_set = dataset['metadata']['utc']
noise_set = dataset['metadata']['noise_state']
freq = file['data']['beam_0']['band_SB0']['frequency'][::12]/1e6

t_set_plt = Time(t_set, format='isot',scale='utc').mjd

In [ ]:
fig1, axs = plt.subplots(1,1,figsize=(18,6))

axs.scatter(t_set_plt,az_set,s=1)

axs.tick_params(axis="x")
axs.tick_params(axis="y")
#axs.set_xlim(t_set_plt[0]+0.08,t_set_plt[-1]+0.01)
axs.set_ylim(-10,370)
axs.set_xlabel('Time (UTC)',fontsize=14)
#axs.set_ylabel('RA (hr)',fontsize=14)
#axs.set_title('Survey phase '+surveyphase+' day '+day,fontsize=14)
axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
axs.fmt_xdata = mdates.DateFormatter('%H:%M:%S')
axs.grid()